In [14]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [15]:
from PIL import Image
import numpy as np

In [17]:
cwd = os.getcwd()
os.chdir(cwd)
print(os.listdir("../datasets/datasetsAugmented/"))

['DatasetCartoon', 'Datasetheroicfantasy', 'DatasetsagaIntergalactique', 'Datasetsuperheroes', 'DatasetVideoGamingCharacters']


In [15]:
'''
#Data augmentation
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

url = "C:/Users/arnaud.baleh/Desktop/dataIA/projetFilRouge/E1_projetFilRougeDl/datasets/VideoGamingCharacters/"
for path in os.listdir(url):        
    img = load_img('C:/Users/arnaud.baleh/Desktop/dataIA/projetFilRouge/E1_projetFilRougeDl/datasets/VideoGamingCharacters/' + path)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir='datasets/datasetsAugmented/DatasetVideoGamingCharacters', save_prefix='VideoGamingCharacters', save_format='png'):
        i += 1
        if i > 20:
            break  # otherwise the generator would loop indefinitely
'''

In [18]:
path_characters = []
train_path_characters = "C:/Users/arnaud.baleh/Desktop/dataIA/projetFilRouge/E1_projetFilRougeDl/datasets/datasetsAugmented/DatasetCartoon/"
for path in os.listdir(train_path_characters):
    if '.png' in path:
        path_characters.append(os.path.join(train_path_characters, path))

lenghtSplit = round(len(path_characters) * 0.95)
new_path=path_characters[0:lenghtSplit]

In [19]:
images = [np.array(Image.open(path).resize((32,32))) for path in path_characters]

for i in range(len(images)):
    images[i] = ((images[i] - images[i].min())/(255 - images[i].min()))
    #images[i] = images[i]*2-1  #decommenter si fonction d activation est tanh pour la dernière couche du generateur

images = np.array(images)

train_data=images

In [20]:
print(train_data.shape)

(9129, 32, 32, 3)


In [22]:
print(type(images))

<class 'numpy.ndarray'>


In [23]:
np.shape(images)

(9129, 32, 32, 3)

In [28]:
tailleInitiale = len(images)
print(tailleInitiale)

9129


In [34]:
'''
from numpy import asarray
from numpy import savetxt
# load numpy array from csv file
from numpy import loadtxt
#Pour sauvegarder un fichier numpy dans un csv file, il faut le redimensionner en 1D
'''

In [35]:
#imagesReshape1D = images.reshape(-1)

In [36]:
#np.shape(imagesReshape1D)

(28044288,)

In [37]:
# save to csv file
#savetxt('csvFiles/dataCartoon.csv', imagesReshape1D, delimiter=',')

In [38]:
# load array
#data = loadtxt('csvFiles/dataCartoon.csv', delimiter=',')
# print the array
#print(data)

[0.96774194 1.         0.97177419 ... 0.9254902  0.67058824 0.56470588]


In [39]:
#imagesReshape4D = data.reshape(tailleInitiale,32,32,3)

In [40]:
#np.shape(imagesReshape4D)

(9129, 32, 32, 3)

In [ ]:
class Discriminator:
    def __init__(self,L,l,C):
        self.__L = L
        self.__l = l
        self.__C = C
        self.__in_shape = (self.__L,self.__l,self.__C)
    
    def build_model(self):
        model = Sequential()
        # normal
        model.add(Conv2D(64, (3,3), padding='same', input_shape=self.__in_shape))
        model.add(LeakyReLU(alpha=0.2))
        # reduction echantillon
        model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # reduction echantillon
        model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # reduction echantillon
        model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # classification
        model.add(Flatten())
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        # compiler modele
        opt = Adam(lr=0.0002, beta_1=0.5)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model
    
class Generator:
    def __init__(self,latent_dim):
        self.__latent_dim = latent_dim
    
    def build_model(self):
        model = Sequential()
        # 4x4 image
        n_nodes = 256 * 4 * 4
        model.add(Dense(n_nodes, input_dim=self.__latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Reshape((4, 4, 256)))
        # agrandissement echantillon à 8x8
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # agrandissement echantillon à 16*16
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # agrandissement echantillon à 32x32
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # couche de sortie
        model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
        return model
 
    # defini une combinaison du generateur et discriminateur, pour mettre à jour le generateur
def define_gan(g_model, d_model):
    # rendre les poids du discriminateur non entrainable
    d_model.trainable = False
    # les connecter
    model = Sequential()
    # ajout generateur
    model.add(g_model)
    # ajout discriminator
    model.add(d_model)
    # compiler le modele
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model
 

def load_real_samples():
    
    # load cifar10 dataset
    #(trainX, _), (_, _) = load_data()
    trainX = train_data
  
    # convertir d unsigned ints vers float32
    X = trainX.astype('float32')
    # changer l echelle de valeur [0,255] vers [-1,1]
    X = (X - 127.5) / 127.5
    return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y
 
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i])
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()
 
    # evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)
 
    # train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=32):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
 
    # size of the latent space
latent_dim = 100
# create the discriminator
d_model = Discriminator(32,32,3)
d_model = d_model.build_model()
# create the generator
g_model = Generator(latent_dim)
g_model = g_model.build_model()
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/285, d1=0.671, d2=0.695 g=0.692
>1, 2/285, d1=0.471, d2=0.697 g=0.690
>1, 3/285, d1=0.313, d2=0.705 g=0.682
>1, 4/285, d1=0.166, d2=0.724 g=0.665
>1, 5/285, d1=0.077, d2=0.764 g=0.631
>1, 6/285, d1=0.039, d2=0.827 g=0.586
>1, 7/285, d1=0.031, d2=0.937 g=0.547
>1, 8/285, d1=0.051, d2=1.015 g=0.540
>1, 9/285, d1=0.107, d2=1.047 g=0.546
>1, 10/285, d1=0.200, d2=1.056 g=0.571
>1, 11/285, d1=0.291, d2=1.014 g=0.611
>1, 12/285, d1=0.382, d2=0.900 g=0.670
>1, 13/285, d1=0.459, d2=0.846 g=0.691
>1, 14/285, d1=0.486, d2=0.820 g=0.710
>1, 15/285, d1=0.501, d2=0.789 g=0.730
>1, 16/285, d1=0.511, d2=0.756 g=0.775
>1, 17/285, d1=0.549, d2=0.788 g=0.754
>1, 18/285, d1=0.529, d2=0.762 g=0.750
>1, 19/285, d1=0.571, d2=0.767 g=0.765
>1, 20/285, d1=0.595, d2=0.785 g=0.755
>1, 21/285, d1=0.581, d2=0.753 g=0.751
>1, 22/285, d1=0.635, d2=0.745 g=0.736
>1, 23/285, d1=0.660, d2=0.791 g=0.702
>1, 24/285, d1=0.657, d2=0.752 g=0.710
>1, 25/285, d1=0.677, d2=0.755 g=0.723
>1, 26/285, d1=0.699, d2=0.755 g=0

In [ ]:
'''
# exemple de generation d'une image pour un point specific dans l'espace latent
from keras.models import load_model
from numpy import asarray
from matplotlib import pyplot
# chargement modele
model = load_model('generator_model_200.h5')
# all 0s
vector = asarray([[0.75 for _ in range(100)]])
# generation de l'image
X = model.predict(vector)
# mise à l'échelle [-1,1] to [0,1]
X = (X + 1) / 2.0
# affichage du resultat
pyplot.imshow(X[0, :, :])
pyplot.show()
'''